#Databricks notebook: Bronze - Focos de Queimadas (Diário)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import datetime

spark = SparkSession.builder.getOrCreate()

In [0]:
# ==========================
# 🔹 Parâmetro do Job
# ==========================
# Exemplo: data_ref_carga = "2025-10-02"
dbutils.widgets.text("data_ref_carga", "")
data_ref_carga = dbutils.widgets.get("data_ref_carga")

if not data_ref_carga:
    raise ValueError("❌ Parâmetro 'data_ref_carga' não informado (ex: 2025-10-02)")

print(f"📅 Data de referência: {data_ref_carga}")

In [0]:
# ==========================
# 🔹 Configurações
# ==========================
catalog = "amazonia_catalog"
schema = "b_inep"
table_name = "d_foco_queim_inc"

path_raw = "/Volumes/amazonia_catalog/raw/raw_inpe"
file_name = f"focos_diario_br_{data_ref_carga.replace('-', '')}.csv"
file_path = f"{path_raw}/{file_name}"

print(f"📂 Procurando arquivo: {file_path}")

In [0]:
# ==========================
# 🔹 Verifica se o arquivo existe
# ==========================
try:
    files = [f.path for f in dbutils.fs.ls(path_raw) if file_name in f.name]
except Exception as e:
    raise FileNotFoundError(f"❌ Erro ao acessar o volume RAW: {e}")

if not files:
    raise FileNotFoundError(f"⚠️ Nenhum arquivo diário encontrado para {data_ref_carga}")

In [0]:
# ==========================
# 🔹 Leitura do CSV
# ==========================
df = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(file_path)
)

df = df.withColumn("data_ref_carga", lit(data_ref_carga))

print(f"✅ Linhas lidas: {df.count()}")

In [0]:
# ==========================
# 🔹 Escrita na Tabela Bronze
# ==========================
(
    df.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"data_ref_carga = '{data_ref_carga}'")
    .partitionBy("data_ref_carga")
    .saveAsTable(f"{catalog}.{schema}.{table_name}")
)

print(f"💾 Dados gravados em: {catalog}.{schema}.{table_name}")